In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('../data/train.csv', low_memory=False)
test_df = pd.read_csv('../data/test.csv')
store_df = pd.read_csv('../data/store.csv')

print(train_df.shape, test_df.shape, store_df.shape)

(1017209, 9) (41088, 8) (1115, 10)


In [3]:
train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


# Column Description
1. Store - a unique Id for each store
2. DayOfWeek - the day of the week
3. Date - the date of the sales
4. Sales - the turnover for any given day (Target Variable)
5. Customers - the number of customers on a given day
6. Open - an indicator for whether the store was open: 0 = closed, 1 = open
7. StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
8. SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools



In [4]:
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


# Column Description
1. Store - a unique Id for each store
2. StoreType - differentiates between 4 different store models: a, b, c, d
3. Assortment - describes an assortment level: a = basic, b = extra, c = extended
4. CompetitionDistance - distance in meters to the nearest competitor store
5. CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
6. Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
7. Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
8. PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [5]:
train_df = pd.merge(train_df, store_df, on='Store')
test_df = pd.merge(test_df, store_df, on='Store')

print(train_df.shape, test_df.shape)

(1017209, 18) (41088, 17)


In [6]:
print(train_df.columns, test_df.columns)

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object') Index(['Id', 'Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'],
      dtype='object')


In [7]:
train_df.head(5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [8]:
def get_d_m_y(df):
    df['Date']=pd.to_datetime(df['Date'])
    df['Year']=df.Date.dt.year
    df['Month']=df.Date.dt.month
    df['Day']=df.Date.dt.day
    df['WeekOfYear']=df.Date.dt.isocalendar().week

def get_relative_competion(df):
    print(df.shape)
    df['CompetionOpen'] = 12 * (df['Year'] - df['CompetitionOpenSinceYear']) + (df.Month - df.CompetitionOpenSinceMonth)
    df['CompetionOpen'] = df['CompetionOpen'].apply(lambda x: x if x > 0 else 0).fillna(0)

def check_promo_month(data):
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',              
                 7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    try:
        months = (data['PromoInterval'] or '').split(',')
        if data['Promo2Open'] and month2str[data['Month']] in months:
            return 1
        else:
            return 0
    except Exception:
        return 0
def promo_cols(df):
    # Months since Promo2 was open
    df['Promo2Open'] = 12 * (df.Year - df.Promo2SinceYear) +  (df.WeekOfYear - df.Promo2SinceWeek)*7/30
    df['Promo2Open'] = df['Promo2Open'].map(lambda x: 0 if x < 0 else x).fillna(0) * df['Promo2']
    # Whether a new round of promotions was started in the current month
    df['IsPromo2Month'] = df.apply(check_promo_month, axis=1) * df['Promo2']

train_df_copy = train_df.copy()

get_d_m_y(train_df_copy)
get_relative_competion(train_df_copy)
promo_cols(train_df_copy)

train_df_copy

(1017209, 22)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,WeekOfYear,CompetionOpen,Promo2Open,IsPromo2Month
0,1,5,2015-07-31,5263,555,1,1,0,1,c,...,NaN,NaN,NaN,2015,7,31,31,82.0,0.0,0
1,1,4,2015-07-30,5020,546,1,1,0,1,c,...,NaN,NaN,NaN,2015,7,30,31,82.0,0.0,0
2,1,3,2015-07-29,4782,523,1,1,0,1,c,...,NaN,NaN,NaN,2015,7,29,31,82.0,0.0,0
3,1,2,2015-07-28,5011,560,1,1,0,1,c,...,NaN,NaN,NaN,2015,7,28,31,82.0,0.0,0
4,1,1,2015-07-27,6102,612,1,1,0,1,c,...,NaN,NaN,NaN,2015,7,27,31,82.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,2013-01-05,4771,339,1,0,0,1,d,...,22.0,2012.0,"Mar,Jun,Sept,Dec",2013,1,5,1,0.0,7.1,0
1017205,1115,5,2013-01-04,4540,326,1,0,0,1,d,...,22.0,2012.0,"Mar,Jun,Sept,Dec",2013,1,4,1,0.0,7.1,0
1017206,1115,4,2013-01-03,4297,300,1,0,0,1,d,...,22.0,2012.0,"Mar,Jun,Sept,Dec",2013,1,3,1,0.0,7.1,0
1017207,1115,3,2013-01-02,3697,305,1,0,0,1,d,...,22.0,2012.0,"Mar,Jun,Sept,Dec",2013,1,2,1,0.0,7.1,0


In [9]:
inputs_col=['Store','Promo','SchoolHoliday','CompetitionDistance',
       'Promo2','Year','Month','Day','WeekOfYear', 
        'CompetitionOpen', 'Promo2Open', 'IsPromo2Month',
            'DayOfWeek','StateHoliday','StoreType','Assortment']
target_col='Sales'

In [10]:
num_col=['Store','Promo','SchoolHoliday','CompetitionDistance',
       'Promo2','Year','Month','Day','WeekOfYear', 
        'CompetitionOpen', 'Promo2Open', 'IsPromo2Month']
cat_col=['DayOfWeek','StateHoliday','StoreType','Assortment']